<a href="https://colab.research.google.com/github/JacobeCode/Neural_Network_For_Speech_Recognition/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
!pip install theano

# Main modules import
import keras
import librosa
import tensorflow
import os
import datetime

# Side modules import
import numpy as np
import theano as te
import matplotlib as plt
import pandas as pd

# Needed parts - import
from pathlib import Path
from google.colab import drive
from datetime import datetime
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Reshape, Flatten, LSTM, Conv2D, MaxPooling2D
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.losses import categorical_crossentropy, binary_crossentropy, sparse_categorical_crossentropy
from keras.utils.np_utils import to_categorical
from IPython.display import Audio, display
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, StratifiedKFold

# Root directory for files - for different directory - change here

# Python local version

# root_dir = Path.cwd()
# train_set_dir = os.path.join(Path.cwd(), "Recordings")
# train_set_list = os.listdir(train_set_dir)
# eval_set_dir = os.path.join(Path.cwd(), "Eval_Recordings")
# eval_set_list = os.listdir(eval_set_dir)

# Colab with Drive version
drive.mount('/content/gdrive/')

root_dir = '/content/gdrive/MyDrive/Speech_Recognition'
model_dir = '/content/gdrive/MyDrive/Speech_Recognition/Model'
CSV_dir = '/content/gdrive/MyDrive/Speech_Recognition/CSV'
train_set_dir = '/content/gdrive/MyDrive/Speech_Recognition/Recordings'
train_set_list = os.listdir(train_set_dir)
eval_set_dir = '/content/gdrive/MyDrive/Speech_Recognition/Eval_Recordings'
eval_set_list = os.listdir(eval_set_dir)
os.chdir(root_dir)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [169]:
# Parameters for comfortable changes later
n_mfcc = 20                                     # Number of MFCC coefficients
win_length = 512                                # Length of MFCC window
n_batch_size = 128                              # Batch Size
n_epoch = 10                                    # Number of Epoch's to perform

add_display = 1
loss_function = 'categorical_crossentropy'      # Type of loss function
optimizer = 'Adam'                              # Type of optimizer
metric_type = 'accuracy'                        # Type of testing metric
test_percentage = 0.1                           # How much of data will be selected to test the training

In [141]:
# Data pre-processing - building database
eval_set_base = pd.DataFrame()

os.chdir(eval_set_dir)
MFCC_list = []
records = []
for count, audio in enumerate(eval_set_list):
    data, fs = librosa.load(audio, sr=None)
    data = data/max(np.abs(data))
    for iteration, item in enumerate(data):
        if np.abs(item) < 0.01:
            np.delete(data, iteration)
    MFCC = librosa.feature.mfcc(y=data, sr=fs, n_mfcc=n_mfcc, hop_length=int(win_length/2), win_length=win_length, n_fft=win_length)
    MFCC_list.append(MFCC)
    records.append(audio)
MFCC_temp = []
max_len = max(np.shape(x)[1] for x in MFCC_list)
for item in MFCC_list:
    temp = []
    n_add = max_len - np.shape(item)[1]
    for coeff in item:
      temp.append(np.append(coeff, np.zeros(n_add)))
    MFCC_temp.append(temp)
    del temp
MFCC_list = MFCC_temp
del MFCC_temp

for item in MFCC_list:
    np.reshape(item, (n_mfcc, max_len, 1))

eval_set_base["Record"] = records
eval_set_base["MFCC"] = MFCC_list
os.chdir(root_dir)

del MFCC_list
del records

In [142]:
# Data pre-processing - building database
train_set_base = pd.DataFrame()

os.chdir(train_set_dir)
MFCC_list = []
labels = []
records = []
for count, audio in enumerate(train_set_list):
    data, fs = librosa.load(audio, sr=None)
    data = data/max(np.abs(data))
    for iteration, item in enumerate(data):
        if np.abs(item) < 0.01:
            np.delete(data, iteration)
    MFCC = librosa.feature.mfcc(y=data, sr=fs, n_mfcc=n_mfcc, hop_length=int(win_length/2), win_length=win_length, n_fft=win_length)
    labels.append(audio[6])
    records.append(audio)
    MFCC_list.append(MFCC)
MFCC_temp = []
for item in MFCC_list:
    temp = []
    n_add = max_len - np.shape(item)[1]
    for coeff in item:
      temp.append(np.append(coeff, np.zeros(n_add)))
    MFCC_temp.append(temp)
    del temp
MFCC_list = MFCC_temp
del MFCC_temp

for item in MFCC_list:
    np.reshape(item, (n_mfcc, max_len, 1))

train_set_base["Record"] = records
train_set_base["MFCC"] = MFCC_list
train_set_base["Labels"] = labels
os.chdir(root_dir)

del MFCC_list
del labels
del records

In [143]:
print("Training Recordings database : ")
display(train_set_base)

print("Eval Recordings database : ")
display(eval_set_base)

Training Recordings database : 


,Record,MFCC,Labels
0,AO1M1_2_.wav,"[[-662.9744262695312, -631.0436401367188, -568...",2
1,AO1M1_0_.wav,"[[-612.0942993164062, -573.1663818359375, -417...",0
2,AO1M1_1_.wav,"[[-571.8582763671875, -549.147216796875, -550....",1
3,AO1M1_3_.wav,"[[-595.4745483398438, -432.62554931640625, -26...",3
4,AO1M1_9_.wav,"[[-618.9461669921875, -591.8220825195312, -412...",9
...,...,...,...
215,SW1M1_8_.wav,"[[-590.7490234375, -406.0035705566406, -288.78...",8
216,SW1M1_3_.wav,"[[-614.0647583007812, -439.59356689453125, -23...",3
217,SW1M1_5_.wav,"[[-672.4749145507812, -453.1002502441406, -306...",5
218,SW1M1_7_.wav,"[[-596.5587768554688, -483.9967041015625, -410...",7


Eval Recordings database : 


,Record,MFCC
0,040.wav,"[[-638.1614990234375, -542.1619262695312, -438..."
1,058.wav,"[[-344.8790588378906, -298.9581298828125, -290..."
2,004.wav,"[[-618.7013549804688, -599.5948486328125, -599..."
3,029.wav,"[[-402.5004577636719, -392.03924560546875, -39..."
4,064.wav,"[[-449.2403259277344, -417.07275390625, -429.5..."
...,...,...
655,650.wav,"[[-487.6944274902344, -455.0632629394531, -414..."
656,647.wav,"[[-548.4116821289062, -543.112060546875, -549...."
657,643.wav,"[[-615.047607421875, -604.5725708007812, -602...."
658,639.wav,"[[-609.5606689453125, -601.455322265625, -310...."


In [170]:
model = Sequential()
model.add(Conv2D(16, (3,3), padding="valid", activation="relu", input_shape=(n_mfcc, max_len, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(8, (3,3), padding="valid", activation="relu", input_shape=(n_mfcc/2, max_len/2, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten(input_shape=(n_mfcc/4, max_len/4, 1)))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

adam = Adam(learning_rate=0.001)

if optimizer == 'Adam':
    model.compile(loss=loss_function, metrics=[metric_type, loss_function], optimizer=adam)
elif optimizer == 'SGD':
    model.compile(loss=loss_function, metrics=[metric_type, loss_function], optimizer=sgd)

model.summary()

Model: "sequential_60"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_64 (Conv2D)          (None, 18, 124, 16)       160       
                                                                 
 max_pooling2d_49 (MaxPoolin  (None, 9, 62, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_65 (Conv2D)          (None, 7, 60, 8)          1160      
                                                                 
 max_pooling2d_50 (MaxPoolin  (None, 3, 30, 8)         0         
 g2D)                                                            
                                                                 
 flatten_42 (Flatten)        (None, 720)               0         
                                                                 
 dense_63 (Dense)            (None, 256)             

In [171]:
X = np.array(train_set_base['MFCC'].tolist())
y_true = train_set_base['Labels'].tolist()

label_encoder = LabelEncoder()
y_true = to_categorical(label_encoder.fit_transform(y_true))

acc_per_fold = []
loss_pre_fold = []

X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=test_percentage, random_state=42)

start = datetime.now()
history = model.fit(X_train, y_train, batch_size=n_batch_size, epochs=n_epoch, validation_data=(X_test, y_test), verbose=add_display)
duration = datetime.now() - start

results = model.evaluate(X_test, y_test, verbose=add_display)

del start

Epoch 1/10
2/2 [==============================] - 1s 320ms/step - loss: 10.3242 - accuracy: 0.1061 - categorical_crossentropy: 10.3242 - val_loss: 7.8788 - val_accuracy: 0.0455 - val_categorical_crossentropy: 7.8788
Epoch 2/10
2/2 [==============================] - 0s 44ms/step - loss: 4.5951 - accuracy: 0.2121 - categorical_crossentropy: 4.5951 - val_loss: 4.8292 - val_accuracy: 0.2273 - val_categorical_crossentropy: 4.8292
Epoch 3/10
2/2 [==============================] - 0s 56ms/step - loss: 2.6704 - accuracy: 0.4747 - categorical_crossentropy: 2.6704 - val_loss: 3.5120 - val_accuracy: 0.4545 - val_categorical_crossentropy: 3.5120
Epoch 4/10
2/2 [==============================] - 0s 58ms/step - loss: 1.8154 - accuracy: 0.5455 - categorical_crossentropy: 1.8154 - val_loss: 2.5562 - val_accuracy: 0.5000 - val_categorical_crossentropy: 2.5562
Epoch 5/10
2/2 [==============================] - 0s 41ms/step - loss: 1.2511 - accuracy: 0.6970 - categorical_crossentropy: 1.2511 - val_loss: 1

In [172]:
model.save(model_dir)

In [173]:
model = load_model(model_dir)
X = np.array(eval_set_base['MFCC'].tolist())
predict = model.predict(X)

values_from_prediction = []
for item in predict:
    values_from_prediction.append(np.max(item))
class_predictions = np.argmax(predict, axis=-1)
print(class_predictions)

rd={"file": eval_set_base['Record'],
        "prediction":class_predictions,
      "values":values_from_prediction}
results_to_csv=pd.DataFrame(data=rd)
print(results_to_csv)
os.chdir(CSV_dir)
results_to_csv.to_csv('results.csv', index=False, header = False, sep = ',')
os.chdir(root_dir)

21/21 [==============================] - 0s 2ms/step
[9 8 2 7 9 4 5 2 6 1 8 9 2 6 9 6 4 8 8 9 5 5 6 6 9 5 0 3 7 2 6 4 7 9 6 4 2
 7 1 9 4 9 0 0 6 9 4 9 9 4 6 9 6 9 8 6 6 9 9 4 6 1 2 6 9 9 4 7 9 8 5 6 9 2
 0 1 6 7 5 9 1 3 9 4 7 4 9 9 9 1 9 2 9 6 2 4 1 7 7 4 0 9 5 4 7 9 5 4 1 0 1
 5 4 1 5 5 9 4 4 5 2 1 4 9 4 9 9 9 4 2 1 9 9 7 3 1 3 2 0 0 9 4 5 6 2 5 5 9
 7 9 2 9 5 5 7 6 6 1 1 7 5 1 1 6 7 2 7 9 2 3 2 1 4 1 5 5 9 2 7 2 0 9 3 5 0
 0 7 0 7 3 1 6 4 9 9 9 9 4 4 3 7 9 6 0 7 7 1 9 9 9 4 5 9 2 1 0 9 7 4 4 9 3
 6 6 0 6 6 7 4 4 2 6 9 2 9 6 0 7 6 8 4 7 6 1 2 7 5 2 5 2 0 4 9 4 1 1 6 3 4
 9 2 8 1 4 2 4 9 4 4 8 9 2 8 8 1 7 2 5 8 8 8 2 1 6 0 7 9 0 9 4 1 9 9 2 9 7
 7 0 9 1 3 4 4 9 9 9 0 3 1 6 7 4 2 0 5 6 0 4 9 9 0 2 9 4 4 9 7 2 3 4 7 6 1
 9 9 6 7 5 3 4 7 2 9 4 2 9 5 7 4 9 7 5 4 0 4 4 1 4 9 7 9 5 2 4 6 6 5 8 9 7
 3 9 9 0 2 5 4 1 0 8 0 7 9 6 9 7 9 3 6 9 6 5 5 6 9 0 9 0 6 9 0 7 0 9 1 2 9
 9 7 5 7 8 7 9 4 4 1 8 4 2 3 5 6 7 8 2 9 9 9 5 5 4 9 4 5 9 4 0 1 5 5 7 7 3
 7 7 9 0 4 9 1 0 1 9 2 1 1 1 0 5 1 4 4 4 9 9 2 